In [7]:
import pandas as pd
import numpy as np
#import statsmodels.api as sm

In [8]:
#convert stata data to pandas
stata_data='C:/Users/Ramin/Desktop/Health Analytics/NSDUH_2017.DTA'
df=pd.read_stata(stata_data)
t=df.shape[0]

In [9]:
dfa=df[df['CATAG2']!='1 - 12-17 Years Old']
dfa = dfa[df.suicplan != '85 - BAD DATA Logically assigned']
dfa = dfa[df.suicplan != '94 - DON T KNOW']
dfa = dfa[df.suicplan != '97 - REFUSED']
dfa = dfa[df.suicplan != '98 - BLANK']

C:\ProgramData\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\ProgramData\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\ProgramData\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [12]:
dfa.groupby(['suicplan']).size()

suicplan
1 - Yes                               845
2 - No                               1740
85 - BAD DATA Logically assigned        0
94 - DON T KNOW                         0
97 - REFUSED                            0
98 - BLANK                              0
99 - LEGITIMATE SKIP                39652
dtype: int64

In [14]:
dfa.loc[df.suicplan=='1 - Yes', 'suicide'] = 'plan'
dfa.loc[df.suicplan=='2 - No', 'suicide'] = 'ideation'
dfa.loc[df.suicplan=='99 - LEGITIMATE SKIP', 'suicide'] = 'population'

In [16]:
dfa.groupby('irpnrnmrec').size()

irpnrnmrec
1 - Within the past 30 days                               648
2 - More than 30 days ago but within the past 12 mos     1596
3 - More than 12 months ago                              2953
9 - NEVER MISUSED PAIN RELIEVERS                        37040
dtype: int64

In [17]:
dfa.loc[df.irpnrnmrec=='1 - Within the past 30 days', 'misuse'] = 'last year'
dfa.loc[df.irpnrnmrec=='2 - More than 30 days ago but within the past 12 mos', 'misuse'] = 'last year'
# dfa.loc[df.irpnrnmrec=='3 - More than 12 months ago', 'misuse'] = 'sometime'
dfa.loc[df.irpnrnmrec=='9 - NEVER MISUSED PAIN RELIEVERS', 'misuse'] = 'never'

In [22]:
gf=dfa.groupby(['suicide','misuse','amdeyr']).size()

In [24]:
# gf.unstack()[['never','last year']]

In [25]:
gf

suicide     misuse     amdeyr 
ideation    last year  1 - Yes      134
                       2 - No        98
            never      1 - Yes      618
                       2 - No       667
plan        last year  1 - Yes      121
                       2 - No        57
            never      1 - Yes      340
                       2 - No       217
population  last year  1 - Yes      254
                       2 - No      1555
            never      1 - Yes     1951
                       2 - No     33030
dtype: int64

In [78]:
y=dfa.suicide
dummies = pd.get_dummies(y)
y=1-dummies['no']

In [89]:
X=dfa.misuse
X = 1-pd.get_dummies(X)['never']

In [90]:
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 49.655972
         Iterations 6


/Users/mahdimoqri/miniconda3/envs/myenv/lib/python3.7/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


                          Results: Logit
Model:              Logit            Pseudo R-squared: inf         
Dependent Variable: no               AIC:              4194640.6046
Date:               2019-02-18 17:26 BIC:              4194649.2556
No. Observations:   42237            Log-Likelihood:   -2.0973e+06 
Df Model:           0                LL-Null:          0.0000      
Df Residuals:       42236            LLR p-value:      nan         
Converged:          1.0000           Scale:            1.0000      
No. Iterations:     6.0000                                         
---------------------------------------------------------------------
            Coef.    Std.Err.      z       P>|z|     [0.025    0.975]
---------------------------------------------------------------------
never      -1.8162     0.0400   -45.4164   0.0000   -1.8946   -1.7378



/Users/mahdimoqri/miniconda3/envs/myenv/lib/python3.7/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/Users/mahdimoqri/miniconda3/envs/myenv/lib/python3.7/site-packages/statsmodels/discrete/discrete_model.py:3313: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.llf/self.llnull


In [113]:
dfa['s_yes_no']=dfa['suicide'].apply(lambda x: 0 if x=='no' else 1)
dfa['o_yes_no']=dfa['misuse'].apply(lambda x: 0 if x=='never' else 1)

In [115]:
logit_model=sm.Logit(dfa['s_yes_no'],dfa['o_yes_no'])
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.657663
         Iterations 6
                         Results: Logit
Model:              Logit            Pseudo R-squared: -1.856    
Dependent Variable: s_yes_no         AIC:              55557.4273
Date:               2019-02-18 18:17 BIC:              55566.0784
No. Observations:   42237            Log-Likelihood:   -27778.   
Df Model:           0                LL-Null:          -9725.6   
Df Residuals:       42236            LLR p-value:      nan       
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
------------------------------------------------------------------
              Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
------------------------------------------------------------------
o_yes_no     -1.8162    0.0400  -45.4164  0.0000  -1.8946  -1.7378



In [116]:
from sklearn.linear_model import LogisticRegression

In [117]:
clf = LogisticRegression(random_state=0, solver='lbfgs',
...                          multi_class='multinomial').fit(X, y)

ValueError: Expected 2D array, got 1D array instead:
array=[0. 0. 0. ... 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.